In [47]:
class Value:
    def __init__(self, value, _children=()):
        self.value = value
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        
    def __repr__(self):
        return "value: {} ; grad: {}".format(self.value, self.grad)
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.value + other.value, (self, other))
        
        def _backward():
            self.grad += out.grad 
            other.grad += out.grad 
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.value * other.value, (self, other))
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, p):
        assert isinstance(p, (float, int))
        out = Value(self.value**p, (self))
        
        def _backward():
            self.grad += (p * out.grad ** (p-1)) * out.grad
        out._backward = _backward
        return out
    
    def __relu__(self):
        out = 0 if self.data < 0 else self.data
        
        def _backward():
            self.grad += out.data > 0 * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        topo = []
        visited = set()
        def topological_sort(node):
            if node not in visited:
                visited.add(node)
                for c in node._prev:
                    topological_sort(c)
                topo.append(node)
        topological_sort(self)
        
        # Apply the chain rule:
        self.grad = 1
        for n in reversed(topo):
            n._backward()
        
        print(self)
    
    
    

In [48]:
a = Value(3)
a

value: 3 ; grad: 0

In [49]:
b = a * 2

In [50]:
a.backward()

value: 3 ; grad: 1


In [51]:
b.backward()

value: 6 ; grad: 1
